## Protocol 1 : Verifying the Performance of an Arduino Board

This protocol aims to verify the temporal precision and stability of an Arduino board by generating a periodic signal (triggers) and analyzing it.

### Prerequisites

**Hardware:**
* An Arduino board (e.g., Uno, Nano, etc. — here: Leonardo)
* A USB cable (micro-USB or USB-C depending on the board)
* A digital oscilloscope or acquisition device (here Analog Discovery 3)

**Software:**
* **Arduino IDE** (version `1.8.19`)
* Acquisition and measurement software (here **Digilent Waveforms**, version `3.24.2`)
* A **Python 3** environment with the necessary libraries (`pandas`, `numpy`, `matplotlib`, `scipy`)

### Procedure

#### 1. Generating Triggers on Arduino

1. **Prepare the Arduino code (.ino)** that generates triggers on a pin (here pin 8), taking the desired period as input.

    > **Arduino Code (.ino) for automatic periodic trigger generation using TimerOne.**
    >
    > ```cpp
    > // Uses the TimerOne library: https://www.pjrc.com/teensy/td_libs_TimerOne.html
    > #include <TimerOne.h>
    >
    > const int triggerPin = 8;       // TTL output on pin 8
    > const unsigned long period_us = 500000;   // Period = 500 ms
    > const unsigned long pulse_width_us = 1000; // Pulse width = 1 ms
    >
    > void setup() {
    >   pinMode(triggerPin, OUTPUT);
    >   digitalWrite(triggerPin, LOW);
    >   Timer1.initialize(period_us);         // Timer triggered every 500 ms
    >   Timer1.attachInterrupt(trigger);      // Function called at each tick
    > }
    >
    > void loop() {
    >   // Timer handles everything
    > }
    >
    > void trigger() {
    >   digitalWrite(triggerPin, HIGH);
    >   delayMicroseconds(pulse_width_us);    // 1 ms pulse
    >   digitalWrite(triggerPin, LOW);
    > }
    > ```

2. **Connect the Arduino board** to the computer via USB.

3. **Open the Arduino IDE** and ensure the board is correctly recognized (`Tools > Board > "your_board_name"`) and the correct port is selected (`Tools > Port`).

    ![Scope settings](../pictures/path_for_choice_of_port.png)

4. **Verify and upload the code** to the board using the dedicated buttons.

    ![Scope settings](../pictures/verify_button_ide_arduino.png)  
    ![Scope settings](../pictures/upload_button_ide_arduino.png)

5. Wait until the message `Done uploading.` appears at the bottom of the window.

    ![Scope settings](../pictures/done_uploading_ide_arduino.png)

The board should now be generating triggers on the specified pin.


#### 2. Measuring the Triggers

1. **Connect the oscilloscope** to the Arduino:
   * Connect the ground probe (GND) to one of the Arduino’s **GND** pins.
   * Connect Channel 1 to **pin 8** of the Arduino.

2. **Launch the Waveforms software** (or equivalent) and click on **`Scope`**.

    ![Scope settings](../pictures/welcome_page_waveforms.png)  
    ![Scope settings](../pictures/scope_opened.png)

3. Click **`Run`** to start the acquisition. You should observe square pulses.

    ![Scope settings](../pictures/acquisition_triggers_peaks.png)

4. You can inspect the signal (period, amplitude, duty cycle) visually by **adjusting the axis scales**.

    ![Scope settings](../pictures/acquisition_characteristics_one_channel.png)  
    ![Scope settings](../pictures/acquisition_triggers.png)


#### 3. Recording the Signal

1. Click the **`Record`** button.

    ![Scope settings](../pictures/record_button.png)

2. Configure the recording settings:
   * **File format:** `.csv`
   * **Sampling rate:** `10 kHz`
   * **Recording duration:** `20 s`
   * **Save location and filename**

    ![Scope settings](../pictures/choice_of_recording_characteristics_two_channels.png)

3. Select the correct channel (1 or 2) then start the recording. You now have a `.csv` file with the measured signal.


#### 4. Signal Analysis

1. **Prepare the Python script**. It should read the `.csv` file, detect peaks, compute intervals (min, max, mean), and display a histogram.

    > **Python Code (.py) for analyzing trigger regularity from a CSV file**
    >
    > ```python
    > import numpy as np
    > import matplotlib
    > matplotlib.use('Agg')
    > import matplotlib.pyplot as plt
    > from scipy.signal import find_peaks
    > import os
    >
    > FILE_NAME = 'triggers_arduino_better_500ms.csv'
    > SIGNAL_COLUMN = 0
    > SAMPLING_RATE = 10000
    > MIN_DISTANCE_MS = 1.0
    >
    > def analyze_and_plot_csv(file_name):
    >     try:
    >         data = np.loadtxt(file_name, delimiter=',')
    >         print(f"--- Analyzing CSV: {file_name} ---")
    >         if data.ndim == 1:
    >             signal = data
    >         else:
    >             signal = data[:, SIGNAL_COLUMN]
    >         abs_signal = np.abs(signal)
    >         threshold = 0.5 * np.max(abs_signal)
    >         min_distance_samples = int(MIN_DISTANCE_MS * SAMPLING_RATE / 1000)
    >         peaks, _ = find_peaks(abs_signal, height=threshold, distance=min_distance_samples)
    >         intervals = np.diff(peaks) / SAMPLING_RATE * 1000
    >         print(f"Mean: {np.mean(intervals):.2f} ms, Min: {np.min(intervals):.2f} ms, Max: {np.max(intervals):.2f} ms")
    >         plt.hist(intervals, bins=50, edgecolor='black')
    >         plt.axvline(np.mean(intervals), color='red', linestyle='--', label='Mean')
    >         plt.title("Interval Distribution")
    >         plt.legend()
    >         plt.savefig("histogram_output.png")
    > ```

2. **Run the script**.

    ![Scope settings](../pictures/analyze_and_draw_triggers_csv.png)

3. The terminal will show the characteristics, and a histogram will be saved to your folder.

    ![Scope settings](../pictures/command_results_triggers_csv.png)  
    ![Scope settings](../pictures/results_plot_arduino.png)

### Going Further

Repeat the protocol with different periods in your Arduino code to further explore the board’s timing performance.
